# SageMaker built-in BlazingText(Unsupervised) example

1. [Introduction](#Introduction)  
2. [Development Environment and Data Preparation](#Development-Environment-and-Data-Preparation)
    1. [Setup](#Setup)  
    2. [Data Preparation](#Data-Preparation)    
    3. [(Optinal)Use the latest wikipedia](#(Optinal)Use-the-latest-wikipedia)
3. [Training the BlazingText model for generating word vectors](#Training-the-BlazingText-model-for-generating-word-vectors)  
    1. [Create BlazingText Container](#Create-BlazingText-Container)  
    2. [Set Hyperparameters](#Set-Hyperparameters)   
    3. [Training](#Training)
4. [Hosting / Inference](#Hosting-/-Inference)
    1. [Use JSON format for inference](#Use-JSON-format-for-inference)
    2. [Evaluation](#Evaluation)
    3. [Model Artifacts for the Word2Vec Algorithm](#Model-Artifacts-for-the-Word2Vec-Algorithm)
5. [Stop / Close the Endpoint](#Stop-/-Close-the-Endpoint)

## Introduction

Word2Vecは、教師なし学習を用いて、大規模なコーパス内の単語のベクトル表現を生成するためのポピュラーなアルゴリズムです。    
生成されたベクトルは、対応する単語間の意味的な関係を捉えていることが示されており、感情分析、固有表現抽出、機械翻訳など、多くの自然言語処理（NLP）タスクで使用されています。    

SageMaker BlazingTextは、以下の環境でのWord2Vecの効率的な実装を提供しています。

- 単一のCPUインスタンス
- 単一のGPUインスタンス上でのマルチGPU - P2 or P3インスタンス
- 複数のCPUインスタンス(分散学習)

ビルトインアルゴリズムを使用する場合、学習とデプロイに関連するコードのほとんどを開発者が意識する必要がなくなる点も利点となります。    
このノートブックでは、BlazingTextで複数のCPUインスタンスを使ったword2vecの分散学習がどのように使用できるかを示します。

## Development Environment and Data Preparation

## Setup

- モデルデータの保存に使用するS3バケットとプレフィックス、およびトレーニングデータが置かれている場所はノートブックインスタンス、トレーニングインスタンス、およびホスティングインスタンスと同じリージョン内にある必要があります。バケットを指定しない場合、SageMaker SDKは同じリージョン内にあらかじめ定義された命名規則に従ってデフォルトのバケットを作成します。
- IAM ロール ARN は、SageMaker にデータへのアクセスを与えるために使用されます。SageMaker python SDK の **get_execution_role** メソッドを使用して取得できます。

In [ ]:
import sagemaker
from sagemaker import get_execution_role
import boto3
import json

sess = sagemaker.Session()

role = get_execution_role()
print(role)  # This is the role that SageMaker would use to leverage AWS resources (S3, CloudWatch) on your behalf

region = boto3.Session().region_name

output_bucket = sess.default_bucket()  # Replace with your own bucket name if needed
print(output_bucket)
output_prefix = "sagemaker/DEMO-blazingtext-text8"  # Replace with the prefix under which you want to store the data if needed

data_bucket = f"sagemaker-sample-files"  # Replace with the bucket where your data is located
data_prefix = "datasets/text/text8/text8"

## Data Preparation

BlazingTextでは、スペースで区切られたトークンを含む、1行1文の前処理済みのテキストファイルを想定しています。    
このノートブックでは、[日本語版text8](https://github.com/Hironsan/ja.text8)データセット(100MB)を使用します。

In [ ]:
!wget https://s3-ap-northeast-1.amazonaws.com/dev.tech-sketch.jp/chakki/public/ja.text8.zip
!unzip ja.text8.zip

## (Optinal)Use the latest wikipedia

https://dumps.wikimedia.org/jawiki を使用して最新のwikipedeaでデータセットを作成します。

使用するインスタンスタイプにも依存しますがこの作業には約1h時間程度かかります。    
[日本語版text8コーパスを作って分散表現を学習する](https://hironsan.hatenablog.com/entry/japanese-text8-corpus)を参考に作成しています。

**_Note:_ 3GBを超えるデータをダウンロードします。ストレージの空き容量に注意してください**

In [ ]:
!wget https://dumps.wikimedia.org/jawiki/20210720/jawiki-20210720-pages-articles.xml.bz2

In [ ]:
!pip install wikiextractor

In [ ]:
!python -m wikiextractor.WikiExtractor -o extracted jawiki-20210720-pages-articles.xml.bz2

In [ ]:
!pip install mecab-python3
!pip install unidic
!python -m unidic download

In [ ]:
!python process.py

In [ ]:
!python tokenize_and_sampling.py

In [ ]:
f = open("ja.text8")
words = f.read().split()
print('総単語数: ', len(words))
print('異なり語数: ', len(set(words)))

## Upload data to Amazon S3 bucket

用意したデータをS3にアップロードし、モデルのアーティファクトが保存されるS3の出力場所を設定します。

In [ ]:
s3_client = boto3.client("s3")
s3_client.upload_file("ja.text8", output_bucket, output_prefix + "/train")

s3_train_data = f"s3://{output_bucket}/{output_prefix}/train"
s3_output_location = f"s3://{output_bucket}/{output_prefix}/output"

In [ ]:
print(s3_train_data)
print(s3_output_location)

## Training the BlazingText model for generating word vectors

[Word2Vec](https://arxiv.org/pdf/1301.3781.pdf)のオリジナルの実装と同様に、SageMaker BlazingText はネガティブサンプリングを使用して、continuous bag-of-words (CBOW) および skip-gramの効率的な実装をCPUおよびGPU上で提供します。

GPU実装では、高度に最適化されたCUDAカーネルを使用しています。詳細は、[*BlazingText: Scaling and Accelerating Word2Vec using Multiple GPU*](https://dl.acm.org/citation.cfm?doid=3146347.3146354)を参照してください。 

BlazingTextは`CBOW`モードと`Skip-gram`モードでサブワード埋め込みの学習をサポートしています。これにより、BlazingTextは、この[notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/blazingtext_word2vec_subwords_text8/blazingtext_word2vec_subwords_text8.ipynb)で示されているように、Out-Of-Vocabulary(OOV)のない単語ベクトルを生成することができます。

SageMaker BlazingTextでは、CBOWやSkip-gramの他に、効率的なミニバッチや行列-行列演算（[BLAS Level 3 routines](https://software.intel.com/en-us/mkl-developer-reference-fortran-blas-level-3-routines)）を使用した`Batch Skipgram`モードもサポートしています。このモードでは、複数のCPUノードに分散してword2vecの学習を行うことができ、1秒間に数億語を処理するword2vec計算をほぼリニアにスケールアップすることができます。詳細は[*Parallelizing Word2Vec in Shared and Distributed Memory*](https://arxiv.org/pdf/1604.04661.pdf)を参照してください。

BlazingTextは、テキスト分類の**supervised**モードもサポートしています。BlazingTextは、FastTextテキスト分類器を拡張し、カスタムCUDAカーネルを使用してGPUアクセラレーションを活用しています。このモデルは、マルチコアCPUやGPUを使って数分で10億語以上の単語を学習することができ、最先端の深層学習テキスト分類アルゴリズムと同等のパフォーマンスを実現しています。詳細については、[algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext.html)または[the text classification notebook](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/introduction_to_amazon_algorithms/blazingtext_text_classification_dbpedia/blazingtext_text_classification_dbpedia.ipynb)をご参照ください。

要約すると、BlazingTextでは以下のモードが、異なるタイプのインスタンスでサポートされています。


|          Modes         	| cbow (supports subwords training) 	| skipgram (supports subwords training) 	| batch_skipgram 	| supervised |
|:----------------------:	|:----:	|:--------:	|:--------------:	| :--------------:	|
|   Single CPU instance  	|   ✔  	|     ✔    	|        ✔       	|  ✔  |
|   Single GPU instance  	|   ✔  	|     ✔    	|         -       	|  ✔ (Instance with 1 GPU only)  |
| Multiple CPU instances 	|     - 	|        -  	|        ✔       	|  -   | |

ここでは、2台の`c5.2xlargeインスタンス`で`batch_skipgram`モードを使用して、*ja.text8*データセットで単語ベクトルを学習するためのリソース構成とハイパーパラメータを定義します。

### Create BlazingText Container

In [ ]:
region_name = boto3.Session().region_name

In [ ]:
container = sagemaker.image_uris.retrieve("blazingtext", region_name)
print(f"Using SageMaker BlazingText container: {container} ({region_name})")

In [ ]:
bt_model = sagemaker.estimator.Estimator(
    container,
    role,
    instance_count=2,
    instance_type="ml.c5.2xlarge",
    volume_size=5,
    max_run=360000,
    input_mode="File",
    output_path=s3_output_location,
    sagemaker_session=sess,
)

### Set Hyperparameters

ハイパーパラメータ については[algorithm documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/blazingtext_hyperparameters.html)を参照してください。

In [ ]:
bt_model.set_hyperparameters(
    mode="batch_skipgram",
    epochs=5,
    min_count=5,
    sampling_threshold=0.001,
    learning_rate=0.05,
    window_size=5,
    vector_dim=100,
    negative_samples=5,
    batch_size=11,  #  = (2*window_size + 1) (Preferred. Used only if mode is batch_skipgram)
    evaluation=False,  # Perform similarity evaluation on WS-353 dataset at the end of training
    subwords=False # Subword embedding learning is not supported by batch_skipgram
)

In [ ]:
train_data = sagemaker.inputs.TrainingInput(
    s3_train_data,
    distribution="FullyReplicated",
    content_type="text/plain",
    s3_data_type="S3Prefix",
)
data_channels = {"train": train_data}

### Training

`Estimator`オブジェクトがあり、このオブジェクトのハイパーパラメータを設定し、データチャネルをアルゴリズムにリンクしています。あとは、アルゴリズムを学習するだけです。次のコマンドは、アルゴリズムを学習します。アルゴリズムの学習には、いくつかの手順が含まれます。

1. `Estimator`クラスでリクエストしたインスタンスがプロビジョニングされ、適切なライブラリでセットアップされます。
2. データがチャネルからインスタンスにダウンロードされます。
3. トレーニングジョブが開始されます。

データのサイズによっては、プロビジョニングとデータのダウンロードに時間がかかります。したがって、トレーニングジョブのトレーニングログの取得を開始するまでに数分かかる場合があります。

ジョブが完了すると、「Job complete」メッセージが出力されます。トレーニングされたモデルは、Estimatorで `output_path`として設定されたS3バケットに保存されています。

In [ ]:
bt_model.fit(inputs=data_channels, logs=True)

## Hosting / Inference

トレーニングジョブが完了すると、モデルをAmazonSageMakerリアルタイムホストエンドポイントとしてデプロイできます。    
これにより、モデルから予測（または推論）を行うことができます。 トレーニングに使用したのと同じタイプのインスタンスでホストする必要はないことに注意してください。 インスタンスエンドポイントは長期間稼働するため、推論にはより安価なインスタンスを選択することをお勧めします。

In [ ]:
bt_endpoint = bt_model.deploy(initial_instance_count=1, instance_type="ml.m5.xlarge")

### Use JSON format for inference

payloadには、キーが"**instances**"である単語のリストが含まれている必要があります。 BlazingTextはコンテンツタイプ `application / json`をサポートします。

期待する挙動として、エンドポイントは各単語のn次元ベクトル（nはハイパーパラメーターで指定されたvector_dim）をレスポンスします。 単語がトレーニングデータセットにない場合、モデルはゼロベクトルを返します。

In [ ]:
words = ["日本", "五輪"]
payload = {"instances": words}

response = bt_endpoint.predict(
    json.dumps(payload),
    initial_args={"ContentType": "application/json", "Accept": "application/json"},
)

vecs = json.loads(response)
print(vecs)

### Evaluation

学習した単語ベクトルをダウンロードし、[t-SNE](https://en.wikipedia.org/wiki/T-distributed_stochastic_neighbor_embedding)を使用して視覚化してみましょう。

In [ ]:
s3 = boto3.resource("s3")

key = bt_model.model_data[bt_model.model_data.find("/", 5) + 1 :]
s3.Bucket(output_bucket).download_file(key, "model.tar.gz")

`model.tar.gz`を解凍して`vectors.txt`を使用します。

BlazingTextでは、モデルアーティファクトは、単語からベクトルへのマッピングを含む`vectors.txt`とBlazingTextがホスティング、推論に使用するバイナリファイル`vectors.bin`で構成されます。

In [ ]:
!tar -xvzf model.tar.gz

In [ ]:
import numpy as np
from sklearn.preprocessing import normalize

# Read the 400 most frequent word vectors. The vectors in the file are in descending order of frequency.
num_points = 400

first_line = True
index_to_word = []

with open("vectors.txt", "r") as f:
    for line_num, line in enumerate(f):
        if first_line:
            dim = int(line.strip().split()[1])
            word_vecs = np.zeros((num_points, dim), dtype=float)
            first_line = False
            continue
        line = line.strip()
        word = line.split()[0]
        vec = word_vecs[line_num - 1]
        for index, vec_val in enumerate(line.split()[1:]):
            vec[index] = float(vec_val)
        index_to_word.append(word)
        if line_num >= num_points:
            break
word_vecs = normalize(word_vecs, copy=False, return_norm=False)

In [ ]:
from sklearn.manifold import TSNE

tsne = TSNE(perplexity=40, n_components=2, init="pca", n_iter=10000, random_state=42)
two_d_embeddings = tsne.fit_transform(word_vecs[:num_points])
labels = index_to_word[:num_points]

In [ ]:
# 日本語対応
!pip install japanize-matplotlib

In [ ]:
from matplotlib import pylab
import japanize_matplotlib

%matplotlib inline


def plot(embeddings, labels):
    pylab.figure(figsize=(20, 20))
    for i, label in enumerate(labels):
        x, y = embeddings[i, :]
        pylab.scatter(x, y)
        pylab.annotate(
            label, xy=(x, y), xytext=(5, 2), textcoords="offset points", ha="right", va="bottom"
        )
    pylab.show()


plot(two_d_embeddings, labels)

### Model Artifacts for the Word2Vec Algorithm

`vector.txt`は、GensimやSpacyなどの他のツールと互換性のある形式でベクトルを保存します。 たとえば、Gensimユーザーは、次のコマンドを実行して、`vectors.txt`ファイルをロードできます。

In [ ]:
!pip install gensim

In [ ]:
from gensim.models import KeyedVectors

word_vectors = KeyedVectors.load_word2vec_format('vectors.txt', binary=False)

In [ ]:
word_vectors.most_similar(['日本'])

In [ ]:
word_vectors.most_similar(positive=['フランス', '東京'], negative=['パリ'])

In [ ]:
word_vectors.doesnt_match("東京 神奈川 千葉 埼玉 香港".split())

## Stop / Close the Endpoint

最後に、ノートブックを閉じる前にエンドポイントを削除する必要があります。

In [ ]:
sess.delete_endpoint(bt_endpoint.endpoint_name)